In [2]:
import os
from pathlib import Path
import warnings
import numpy as np
import pandas as pd
import torch
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data.encoders import NaNLabelEncoder, TorchNormalizer
from pytorch_forecasting.metrics import QuantileLoss
from pytorch_forecasting.data import GroupNormalizer
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger, CSVLogger

warnings.filterwarnings("ignore")

warnings.filterwarnings("ignore")

# ==========================================
# 1. AYARLAR VE SABİTLER
# ==========================================
TRAIN_PATH = "train_data.pkl"
VAL_PATH   = "val_data.pkl"
TEST_PATH  = "test_data.pkl"

BATCH_SIZE = 64
MAX_ENCODER_LENGTH = 24    # Geçmişe bakış (ör: 24 ay)
MAX_PREDICTION_LENGTH = 6  # Gelecek tahmini (ör: 6 ay)
LEARNING_RATE = 0.03
MAX_EPOCHS = 30
NUM_WORKERS = 0            # Windows'ta 0 olması önerilir

In [3]:
# ==========================================
# 2. VERİ YÜKLEME VE BİRLEŞTİRME
# ==========================================
print("⏳ Veriler yükleniyor...")
train_source = pd.read_pickle(TRAIN_PATH)
val_source   = pd.read_pickle(VAL_PATH)
test_source  = pd.read_pickle(TEST_PATH)

# Veriyi birleştirip temizliği tek seferde yapalım
train_source["split_type"] = "train"
val_source["split_type"]   = "val"
test_source["split_type"]  = "test"

df = pd.concat([train_source, val_source, test_source], ignore_index=True)

# ==========================================
# 3. VERİ TEMİZLİĞİ VE HAZIRLIK
# ==========================================
print("🧹 Veri temizliği yapılıyor...")

UNK = "UNKNOWN"

# Kategorik temizleme fonksiyonu
def clean_cat(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.strip().str.upper()
    # NaN veya boş stringleri UNK ile doldur
    s = s.replace({"": UNK, "NONE": UNK, "NAN": UNK, "<NA>": UNK, "NA": UNK, "NULL": UNK})
    return s.fillna(UNK)

# Kategorik kolonlar
cat_cols = ["DEPARTMENT_ID_DESC", "SKU_GROUP_ID_DESC"]
for c in cat_cols:
    if c not in df.columns:
        raise KeyError(f"{c} kolonu bulunamadı.")
    df[c] = clean_cat(df[c])

# Tekil bir grup ID oluştur (Modelin serileri ayırt etmesi için)
# Departman ve Ürün grubunu birleştirerek benzersiz bir anahtar yapıyoruz
df["group_id"] = df["DEPARTMENT_ID_DESC"] + "_" + df["SKU_GROUP_ID_DESC"]

# Target (y) temizliği
df["y"] = pd.to_numeric(df["y"], errors="coerce").fillna(0.0).astype("float32")
df.loc[df["y"] < 0, "y"] = 0.0  # Negatif satış önlemi

# Reel Değişkenler (process.py'den gelmesi beklenenler dışında ek temizlik)
real_cols = ["budget_month", "eurtry_month", "inflation", "edu"]
for col in real_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0.0).astype("float32")
        # Enflasyon 100 bazındaysa 1 baza çek (TFT küçük sayıları sever)
        if col == "inflation":
            mask_high = df[col] > 100
            df.loc[mask_high, col] = df.loc[mask_high, col] / 100
        df.loc[df[col] < 0, col] = 0.0
    else:
        # Kolon eksikse hata vermesin, 0 ile doldursun
        df[col] = 0.0

# Veriyi tekrar ayıralım
train_df = df[df["split_type"] == "train"]
val_df   = df[df["split_type"] == "val"]

print(f"✅ Hazır. Train: {len(train_df)}, Val: {len(val_df)}")


# 1. Eğitim setindeki benzersiz grupları al
train_active_groups = train_df["group_id"].unique()

# 2. Validation setini sadece bu grupları içerecek şekilde filtrele
# (Eğitimde olmayan bir grup validation'a girdiyse çıkarılır)
val_df = val_df[val_df["group_id"].isin(train_active_groups)]

print(f"Filtreleme sonrası - Train: {len(train_df)}, Val: {len(val_df)}")

# ==========================================
# 4. DATASET TANIMLAMA (TimeSeriesDataSet)
# ==========================================
# Hedef değişken için normalizer (Log dönüşümü veya Softplus önerilir)
target_normalizer = GroupNormalizer(
    groups=["group_id"],
    transformation="softplus"
)

training = TimeSeriesDataSet(
    train_df,
    time_idx="time_idx",
    target="y",
    group_ids=["group_id"],

    # Statik Kategorikler (Değişmeyenler)
    static_categoricals=["DEPARTMENT_ID_DESC", "SKU_GROUP_ID_DESC"],

    # Bilinen Gelecek Değişkenleri (Zaman + process.py'den gelenler)
    time_varying_known_reals=[
        "time_idx",
        "month_sin",
        "month_cos",
        "budget_month",
        "eurtry_month",
        "inflation",
        "edu"
    ],

    # Bilinmeyen Gelecek (Sadece geçmişte bildiğimiz hedef)
    time_varying_unknown_reals=["y"],

    min_encoder_length=MAX_ENCODER_LENGTH // 2, # Esneklik sağlar
    max_encoder_length=MAX_ENCODER_LENGTH,
    min_prediction_length=1,
    max_prediction_length=MAX_PREDICTION_LENGTH,

    allow_missing_timesteps=True,
    target_normalizer=target_normalizer,
    categorical_encoders={
        "DEPARTMENT_ID_DESC": NaNLabelEncoder(add_nan=True),
        "SKU_GROUP_ID_DESC": NaNLabelEncoder(add_nan=True)
    }
)

# Validation seti (Training parametrelerini miras alır)
validation = TimeSeriesDataSet.from_dataset(training, val_df, predict=True, stop_randomization=True)

# Dataloaderlar
train_dataloader = training.to_dataloader(train=True, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
val_dataloader   = validation.to_dataloader(train=False, batch_size=BATCH_SIZE * 3, num_workers=NUM_WORKERS)


⏳ Veriler yükleniyor...
🧹 Veri temizliği yapılıyor...
✅ Hazır. Train: 11145, Val: 1390
Filtreleme sonrası - Train: 11145, Val: 1384


In [8]:
# ==========================================
# 5. MODEL KURULUMU (TFT)
# ==========================================
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=LEARNING_RATE,
    hidden_size=64,           # Model karmaşıklığı
    lstm_layers=2,
    dropout=0.1,
    output_size=7,            # QuantileLoss default 7 quantiles
    loss=QuantileLoss(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)

print(f"✅ Model oluşturuldu. Parametre sayısı: {tft.size()/1e3:.1f}k")

# ==========================================
# 6. EĞİTİM (TRAINER)
# ==========================================
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath="models/",
    filename="best-checkpoint-{epoch:02d}-{val_loss:.2f}",
    save_top_k=1,
    mode="min"
)

early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=1e-4,
    patience=10,
    verbose=False,
    mode="min"
)

lr_logger = LearningRateMonitor()

# Trainer artık lightning.pytorch üzerinden çalışıyor
trainer = pl.Trainer(
    max_epochs=MAX_EPOCHS,
    accelerator="auto",
    gradient_clip_val=0.1,
    callbacks=[lr_logger, early_stop_callback, checkpoint_callback],
    limit_train_batches=1.0
)

print("🚀 Eğitim başlıyor...")

# Debug kontrolü
if not isinstance(tft, pl.LightningModule):
    # Eğer bu uyarı çıkarsa runtime restart edilmemiş demektir
    print("UYARI: Model sınıfı hala uyuşmuyor! Lütfen pip işlemlerinden sonra runtime'ı restart edin.")

trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: `Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
INFO:lightning.pytorch.utilities.rank_zero:`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
   | Name                               | Type                            | Params
----------------------------------

✅ Model oluşturuldu. Parametre sayısı: 303.4k
🚀 Eğitim başlıyor...


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [5]:
# ==========================================
# 8. TEST VE TAHMİN RAPORLAMA (DÜZELTİLMİŞ)
# ==========================================
print("\n📊 Test süreci ve Raporlama başlıyor...")

# ---------------------------------------------------------
# A. Test Verisinin Hazırlanması
# df değişkeni üst satırlardan kalmadıysa dosyadan tekrar yükle
# ---------------------------------------------------------
if 'df' not in globals():
    # Eğer hafızada df yoksa, dosyadan okuyup filtreleyelim
    print("⚠️ Ana dataframe hafızada bulunamadı, dosyadan yükleniyor...")
    test_source = pd.read_pickle("test_data.pkl")
    test_source["split_type"] = "test"
    # Temizlik fonksiyonları yukarıda tanımlı varsayılıyor, değilse buraya eklenmeli
    # Basitlik için sadece gerekli kolonları alıyoruz
    test_df = test_source.copy()

    # Kategori temizliği (varsa clean_cat fonksiyonu kullanılabilir)
    # Target temizliği
    test_df["y"] = pd.to_numeric(test_df["y"], errors="coerce").fillna(0.0).astype("float32")
    test_df["group_id"] = test_df["DEPARTMENT_ID_DESC"] + "_" + test_df["SKU_GROUP_ID_DESC"]
else:
    # df zaten varsa içinden test kısmını çek
    test_df = df[df["split_type"] == "test"].copy()

# Filtreleme: Modelin eğitim sırasında gördüğü (veya tanımlı olduğu) gruplar
# Sadece validation aşamasında değil, test aşamasında da bilinmeyen gruplar hata verir.
# Bu yüzden training dataset'in encoder'larını kullanıyoruz.
test_dataset = TimeSeriesDataSet.from_dataset(
    training,
    test_df,
    predict=True,
    stop_randomization=True
)

test_dataloader = test_dataset.to_dataloader(train=False, batch_size=BATCH_SIZE * 2, num_workers=0)

# ---------------------------------------------------------
# B. Tahminlerin Üretilmesi
# best_tft: Yukarıdaki (7. Adım) model yükleme kısmından geliyor
# ---------------------------------------------------------
print("🔮 Tahminler üretiliyor...")
raw_predictions = best_tft.predict(
    test_dataloader,
    mode="prediction",
    return_y=True,
    return_x=True,
    trainer_kwargs=dict(accelerator="cpu") # Tahmin sırasında CPU daha güvenli (bellek şişmemesi için)
)

predictions = raw_predictions.output  # Tahminler (Batch x Prediction Length)
actuals = raw_predictions.y[0]        # Gerçek Değerler
x_data = raw_predictions.x            # Meta veriler (Grup id'leri vb.)

# ---------------------------------------------------------
# C. Global Metrikler
# ---------------------------------------------------------
from torchmetrics.functional import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

rmse = torch.sqrt(mean_squared_error(predictions, actuals))
mae = mean_absolute_error(predictions, actuals)
mape = mean_absolute_percentage_error(predictions, actuals)

print(f"\n🏆 GLOBAL TEST METRİKLERİ:")
print(f"   RMSE (Hata Karekök Ort.): {rmse:.4f}")
print(f"   MAE  (Mutlak Hata Ort.):  {mae:.4f}")
print(f"   MAPE (Model Yanılması %): {mape:.2%}")

# ---------------------------------------------------------
# D. Excel Raporu Oluşturma
# ---------------------------------------------------------
print("\n💾 Detaylı Excel raporu hazırlanıyor...")

results = []
n_samples = predictions.shape[0]
pred_len = predictions.shape[1]

# Decoder'daki grup kodlarını çözmek biraz karmaşık olabilir,
# o yüzden sample index üzerinden test_df ile eşleştirme yapacağız.
# Test dataloader sıralı olduğu için indexleri takip edebiliriz.

# Tahmin sonuçlarını düzleştir
flat_preds = predictions.numpy().flatten()
flat_acts = actuals.numpy().flatten()
n_total = len(flat_preds)

# Metrikleri hesaplayıp listeye atalım
for i in range(n_total):
    pred_val = float(flat_preds[i])
    act_val = float(flat_acts[i])

    # Basit Accuracy Formülü: 1 - (|Hata| / |Gerçek|)
    if act_val == 0:
        # Eğer gerçek 0 ise ve biz de 0 tahmin ettiysek %100, yoksa %0
        acc_score = 100.0 if abs(pred_val) < 1e-4 else 0.0
    else:
        diff = abs(pred_val - act_val)
        acc_score = max(0, (1 - (diff / abs(act_val)))) * 100

    results.append({
        "Prediction (Tahmin)": pred_val,
        "Actual (Gerçek)": act_val,
        "Difference (Fark)": pred_val - act_val,
        "Accuracy (%)": acc_score
    })

res_df = pd.DataFrame(results)

# Meta verileri ekleyelim (Departman, Ürün vb.)
# Not: predict(mode="prediction") çıktısı ile orijinal dataframe satırları tam eşleşmeyebilir
# (TimeSeriesDataSet pencereleme yaptığı için).
# Ancak en temiz yöntem, x_data içindeki grup bilgilerini çözmektir. Orası karmaşık olduğu için
# Raporu sadece sayısal değerler ve index bazlı olarak dışarı alıyoruz.

# Daha detaylı eşleştirme için x_data["decoder_target"] kullanılabilirdi ama
# şimdilik sadece sayısal karşılaştırmayı kaydediyoruz.
OUTPUT_FILE = "data/test_predictions_report.xlsx"
res_df.to_excel(OUTPUT_FILE, index_label="Row_ID")

print(f"✅ İşlem Tamamlandı! Rapor kaydedildi: {OUTPUT_FILE}")
print(res_df.head(10))


📊 Test süreci ve Raporlama başlıyor...


KeyError: "Unknown category 'SANZIMAN ISLEME + MONTAJ MÜDÜRLÜGÜ_ELEKTRIK MALZEMELERI' encountered. Set `add_nan=True` to allow unknown categories"